# YOLOv8 Training for Brain Tumor Detection

This notebook trains a YOLOv8 model for brain tumor detection and localization with bounding boxes.

## Overview

YOLOv8 is used for **object detection** - it can detect and localize tumors in MRI images by drawing bounding boxes around them.

**Note:** This requires bounding box annotations. If you don't have annotated data, you'll need to:
1. Use an annotation tool like [LabelImg](https://github.com/tzutalin/labelImg) to create bounding boxes
2. Or use the dataset preparation script which creates placeholder labels (for structure only)

**Note on Augmented Data:** YOLOv8 uses a different dataset structure (YOLO format with images and labels). If you want to use augmented training images, you'll need to:
1. Run `augment_training_data.py` to create augmented images
2. Convert augmented images to YOLO format (images in `data/yolov8/images/train/` and corresponding labels in `data/yolov8/labels/train/`)
3. Update the YAML configuration file to point to the augmented dataset if using a separate augmented YOLO dataset

## Requirements

Make sure you have installed:
```bash
pip install ultralytics
```

Or install from requirements.txt:
```bash
pip install -r requirements.txt
```

## 1. Setup and Imports

In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path

# Check if YOLOv8 dataset is prepared
yolo_data_dir = 'data/yolov8'
yaml_config = 'tumor_data.yaml'

if not os.path.exists(yolo_data_dir):
    print("WARNING: YOLOv8 dataset not found!")
    print("Please run: python prepare_yolo_dataset.py")
    print("Or manually prepare the dataset structure.")
else:
    print(f"YOLOv8 dataset found at: {yolo_data_dir}")

if not os.path.exists(yaml_config):
    print(f"WARNING: Configuration file '{yaml_config}' not found!")
else:
    print(f"Configuration file found: {yaml_config}")

## 2. Dataset Preparation Check

In [ ]:
# Check dataset structure
def check_yolo_dataset():
    """Check if YOLOv8 dataset is properly structured."""
    required_dirs = [
        'data/yolov8/images/train',
        'data/yolov8/images/val',
        'data/yolov8/labels/train',
        'data/yolov8/labels/val'
    ]
    
    all_exist = True
    for dir_path in required_dirs:
        exists = os.path.exists(dir_path)
        status = "OK" if exists else "MISSING"
        print(f"{status}: {dir_path}")
        if exists:
            file_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
            print(f"   -- {file_count} files")
        all_exist = all_exist and exists
    
    return all_exist

dataset_ready = check_yolo_dataset()

if not dataset_ready:
    print("\nWARNING: Dataset not ready! Please run prepare_yolo_dataset.py first.")
else:
    print("\nDataset structure is ready for training!")

## 3. YOLOv8 Model Initialization

In [ ]:
# Initialize YOLOv8 model
# Options: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large), yolov8x.pt (xlarge)
model = YOLO('yolov8n.pt')  # Start with nano for faster training

print("YOLOv8 model initialized")
print(f"Model: yolov8n.pt (nano - smallest, fastest)")
print("\nAvailable model sizes:")
print("  - yolov8n.pt: Nano (smallest, fastest)")
print("  - yolov8s.pt: Small")
print("  - yolov8m.pt: Medium")
print("  - yolov8l.pt: Large")
print("  - yolov8x.pt: XLarge (largest, most accurate)")

## 4. Training Configuration

In [ ]:
# Training hyperparameters
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 640
PATIENCE = 10  # Early stopping patience
MODEL_DIR = 'models/yolov8'

# Create model directory
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Early Stopping Patience: {PATIENCE}")
print(f"  Dataset Config: {yaml_config}")
print(f"  Model Directory: {MODEL_DIR}")

## 5. Start Training

In [ ]:
# Train the model
results = model.train(
    data=yaml_config,           # Path to dataset YAML file
    epochs=EPOCHS,              # Number of training epochs
    imgsz=IMG_SIZE,             # Input image size
    batch=BATCH_SIZE,           # Batch size
    patience=PATIENCE,          # Early stopping patience
    project=MODEL_DIR,         # Project directory
    name='yolov8_tumor_detection',  # Experiment name
    save=True,                  # Save checkpoints
    plots=True                  # Generate training plots
)

print("\nTraining completed!")

## 6. Model Evaluation

In [ ]:
# Evaluate the model on validation set
metrics = model.val()

print("\nValidation Metrics:")
print(f"  mAP@0.5: {metrics.box.map50:.4f}")
print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

## 7. Load Best Model and Test

In [ ]:
# Load the best model from training
best_model_path = os.path.join(MODEL_DIR, 'yolov8_tumor_detection', 'weights', 'best.pt')
best_model = YOLO(best_model_path)

print(f"Loaded best model from: {best_model_path}")

# Test on a sample image (if available)
test_image_path = 'data/yolov8/images/val'
if os.path.exists(test_image_path):
    # Get first image from validation set
    val_images = [f for f in os.listdir(test_image_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if val_images:
        sample_image = os.path.join(test_image_path, val_images[0])
        print(f"\nTesting on sample image: {sample_image}")
        
        # Run prediction
        results = best_model.predict(
            source=sample_image,
            conf=0.25,  # Confidence threshold
            save=True,  # Save prediction results
            project=MODEL_DIR,
            name='predict'
        )
        
        print(f"Prediction saved to: {os.path.join(MODEL_DIR, 'predict')}/")

## 8. Summary

### Training Results:
- Model saved to: `models/yolov8/yolov8_tumor_detection/weights/best.pt`
- Training plots saved in: `models/yolov8/yolov8_tumor_detection/`
- Validation metrics calculated above

### Using the Trained Model:

**Windows:**
```cmd
yolo task=detect mode=predict model=models/yolov8/yolov8_tumor_detection/weights/best.pt source=path/to/image.jpg conf=0.25
```

**Linux:**
```bash
yolo task=detect mode=predict model=models/yolov8/yolov8_tumor_detection/weights/best.pt source=path/to/image.jpg conf=0.25
```

### Important Notes:
- YOLOv8 requires proper bounding box annotations for effective training
- The placeholder labels created by `prepare_yolo_dataset.py` cover the entire image
- For real detection, annotate images with actual tumor bounding boxes using tools like LabelImg
- Training results and plots are saved in the `runs/detect/` directory